In [11]:
import pandas as pd
import numpy as np
# from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import SGDRegressor

In [12]:
# =========================================================
# I. DATA LOADING AND PREPARATION
# =========================================================

california_houses=pd.read_csv("datasets/California_Houses.csv")

# Separate features (X) and target (T)
X = california_houses.drop(columns=['Median_House_Value'])
T = california_houses['Median_House_Value']


# Shuffle the data to ensure randomness
shuffled_data = pd.concat([X, T], axis=1).sample(frac=1, random_state=42).reset_index(drop=True)
X_shuffled = shuffled_data.drop(columns=['Median_House_Value'])
T_shuffled = shuffled_data['Median_House_Value']

# Define the split points (70% train, 15% validation, 15% test)
#raw means before scaling (normalization)
total_rows = X_shuffled.shape[0]
train_end = int(total_rows * 0.7)
validation_end = int(total_rows * 0.85)

# Assign the training data portion (0% to 70%)
X_train_raw = X_shuffled.iloc[:train_end]
T_train = T_shuffled.iloc[:train_end]

# Assign the validation data portion (70% to 85%)
X_validation_raw = X_shuffled.iloc[train_end:validation_end]
T_validation = T_shuffled.iloc[train_end:validation_end]

# Assign the test data portion (85% to 100%)
X_test_raw = X_shuffled.iloc[validation_end:]
T_test = T_shuffled.iloc[validation_end:]


# --- Feature Scaling (Crucial for Gradient Descent) ---

#scaler is used to normalize features , mean =0 , std = 1

# Gradient Descent converges faster when features are on similar scales
# Prevents features with large values from dominating the learning process
# Ensures all features contribute equally to the model
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_raw)

X_validation_scaled = scaler.transform(X_validation_raw)
X_test_scaled = scaler.transform(X_test_raw)


# --- Add Bias Term ---

# Add bias term (column of ones) to all three scaled sets
X_train_with_bias_scaled = np.c_[np.ones((len(X_train_scaled), 1)), X_train_scaled]
X_validation_with_bias_scaled = np.c_[np.ones((len(X_validation_scaled), 1)), X_validation_scaled]
X_test_with_bias_scaled = np.c_[np.ones((len(X_test_scaled), 1)), X_test_scaled]


# --- Reshape Target Vars to col vectors---

T_train_col = T_train.values.reshape(-1, 1)
T_validation_col = T_validation.values.reshape(-1, 1)
T_test_col = T_test.values.reshape(-1, 1)



In [13]:
# =========================================================
# II. DIRECT SOLUTION (NORMAL EQUATION)
# =========================================================

print("="*60)
print("             1. DIRECT SOLUTION (NORMAL EQUATION)")
print("="*60)

# --- 1. Train (Using UN-SCALED features for the Normal Equation) ---
""" 
The Normal Equation doesn't need normalization(scaled features) because
it solves the problem in one step using a closed-form mathematical formula.
It's not iterative, so it doesn't suffer from convergence issues.
"""
X_train_with_bias_raw = np.c_[np.ones((len(X_train_raw), 1)), X_train_raw.values]

# W* = (X_T * X)^-1 * X_T * T
W_direct_sol = np.linalg.pinv(X_train_with_bias_raw.T @ X_train_with_bias_raw) @ X_train_with_bias_raw.T @ T_train_col

# --- 2. Predict on Test Set ---
X_test_with_bias_raw = np.c_[np.ones((len(X_test_raw), 1)), X_test_raw.values]
X_validation_with_bias_raw = np.c_[np.ones((len(X_validation_raw), 1)), X_validation_raw.values]

T_validation_prediction = X_validation_with_bias_raw @ W_direct_sol
T_test_prediction = X_test_with_bias_raw @ W_direct_sol

             1. DIRECT SOLUTION (NORMAL EQUATION)


In [14]:
# =========================================================
# III. GRADIENT DESCENT (BATCH GD)
# =========================================================

print("\n" + "="*60)
print("             2. GRADIENT DESCENT (BATCH GD)")
print("="*60)

"""
    Gradient Descent Algorithm for Linear Regression
    
    Cost Function (MSE):
        J(W) = (1/2m) * Σ(i=1 to m) [h(x^(i)) - y^(i)]²
        where h(x) = W₀ + W₁x₁ + W₂x₂ + ... + Wₙxₙ = X @ W
        my prediction - real thing
        
    Partial Derivatives (Gradients):
        ∂J/∂Wⱼ = (1/m) * Σ(i=1 to m) [(h(x^(i)) - y^(i)) * xⱼ^(i)]
        
        For each weight Wⱼ (j = 0, 1, 2, ..., n):
        - Sum over all m training examples
        - Multiply error by the j-th feature value
        - Average by dividing by m
    
    Update Rule:
        Wⱼ := Wⱼ - learningRate * ∂J/∂Wⱼ
        := means assigned to
    
    Vectorized Form:
        gradient = (1/m) * X^T @ (X @ W - y)
    """

def gradient_descent(X, y, learning_rate=0.01, n_iterations=2000):
    m = len(y)
    # Initialize theta (coefficients) to zeros
    W = np.zeros((X.shape[1], 1))

    for _ in range(n_iterations):
        error = (X @ W) - y
        gradients = (1/m) * X.T @ error
        W = W - learning_rate * gradients

    return W

# --- 1. Train (Using SCALED features) ---
learning_rate = 0.01
n_iterations = 2000
W_gd = gradient_descent(X_train_with_bias_scaled, T_train_col, learning_rate, n_iterations)

# --- 2. Predict on Test Set ---
T_validation_predict_gd = X_validation_with_bias_scaled @ W_gd
T_test_predict_gd = X_test_with_bias_scaled @ W_gd



             2. GRADIENT DESCENT (BATCH GD)


In [15]:
# IV. GRADIENT DESCENT (BATCH GD) - REGULARIZED VERSIONS


# Helper function to calculate Mean Squared Error
def calculate_mse(y_true, y_pred):
    return np.mean((y_pred - y_true)**2)

def ridge_gradient_descent(X, y, lambda_reg, learning_rate=0.01, n_iterations=5000):
    m, n = X.shape
    # Initialize theta (coefficients)
    theta = np.zeros((n, 1))

    for _ in range(n_iterations):
        predictions = X @ theta
        error = predictions - y
        
        # Standard Gradient (1/m * X.T @ error)
        gradients = (1/m) * X.T @ error

        # L2 Regularization Penalty Gradient (2*lambda*w)
        # We create a penalty vector: weights * 2 * lambda_reg
        # Note: Do NOT penalize the intercept/bias term (index 0)
        l2_penalty_gradient = (2 * lambda_reg / m) * theta
        l2_penalty_gradient[0] = 0 # Set penalty for bias term to zero

        # Update theta with the combined gradient
        theta = theta - learning_rate * (gradients + l2_penalty_gradient)
        
    return theta





In [16]:
#lasso regression
def lasso_gradient_descent(X, y, lambda_reg, learning_rate=0.01, n_iterations=5000):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for _ in range(n_iterations):
        predictions = X @ theta
        error = predictions - y
        
        # Standard Gradient (1/m * X.T @ error)
        gradients = (1/m) * X.T @ error

        # L1 Regularization Penalty Gradient (lambda * sign(w))
        # We create a penalty vector: lambda * sign(weights)
        # Note: Do NOT penalize the intercept/bias term (index 0)
        l1_penalty_gradient = (lambda_reg / m) * np.sign(theta)
        l1_penalty_gradient[0] = 0 # Set penalty for bias term to zero
        
        # Update theta with the combined gradient
        theta = theta - learning_rate * (gradients + l1_penalty_gradient)
        
    return theta


In [17]:
# =========================================================
# V. COMPARISON AND EVALUATION Using SciKit
# =========================================================

def compare_metrics(T_true, T_pred_normal, T_pred_gd, model_type):
    """Calculates and compares metrics for both models on a given set."""
    mse_normal = mean_squared_error(T_true, T_pred_normal)
    rmse_normal = np.sqrt(mse_normal)
    r2_normal = r2_score(T_true, T_pred_normal)
    
    mse_gd = mean_squared_error(T_true, T_pred_gd)
    rmse_gd = np.sqrt(mse_gd)
    r2_gd = r2_score(T_true, T_pred_gd)
    
    metrics_df = pd.DataFrame({
        'Metric': ['RMSE', 'R² Score'],
        'Normal Equation': [f'{rmse_normal:,.2f}', f'{r2_normal:.4f}'],
        'Gradient Descent': [f'{rmse_gd:,.2f}', f'{r2_gd:.4f}']
    })
    
    print(f"\n--- Model Performance Comparison on the {model_type} ---")
    print(metrics_df.to_string(index=False))
    return metrics_df

def compare_metrics_all(T_true, T_pred_normal, T_pred_gd, T_pred_sklearn, model_type):
    """Calculates and compares metrics for all three models on a given set."""
    mse_normal = mean_squared_error(T_true, T_pred_normal)
    rmse_normal = np.sqrt(mse_normal)
    r2_normal = r2_score(T_true, T_pred_normal)
    
    mse_gd = mean_squared_error(T_true, T_pred_gd)
    rmse_gd = np.sqrt(mse_gd)
    r2_gd = r2_score(T_true, T_pred_gd)
    
    mse_sklearn = mean_squared_error(T_true, T_pred_sklearn)
    rmse_sklearn = np.sqrt(mse_sklearn)
    r2_sklearn = r2_score(T_true, T_pred_sklearn)
    
    metrics_df = pd.DataFrame({
        'Metric': ['RMSE', 'R² Score'],
        'Normal Equation': [f'{rmse_normal:,.2f}', f'{r2_normal:.4f}'],
        'Manual GD': [f'{rmse_gd:,.2f}', f'{r2_gd:.4f}'],
        'Scikit-Learn SGD': [f'{rmse_sklearn:,.2f}', f'{r2_sklearn:.4f}']
    })
    
    print(f"\n--- Model Performance Comparison on the {model_type} ---")
    print(metrics_df.to_string(index=False))
    return metrics_df

# --- 1. Compare Coefficients ---
feature_names = ['Intercept'] + list(X_train_raw.columns)
coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Normal Eq. (Raw Scale)': W_direct_sol.flatten(),
    'GD (Scaled)': W_gd.flatten()
})

print("\n" + "="*60)
print("           COEFFICIENT COMPARISON")
print("="*60)
print("Note: Normal Eq. weights apply to raw features, GD weights to scaled features.")
print("\n" + coefficients_df.to_string(index=False))


# --- 2. Compare Metrics on Test Set ---
print("\n" + "="*60)
print("         TEST SET METRICS COMPARISON")
print("="*60)
com = compare_metrics(T_test_col, T_test_prediction, T_test_predict_gd, "Test Set")


print("\n" + "="*60)
print("           FINAL COEFFICIENTS TABLE")
print("="*60)
display(coefficients_df)  # or just: coefficients_df (for Jupyter display)



           COEFFICIENT COMPARISON
Note: Normal Eq. weights apply to raw features, GD weights to scaled features.

                 Feature  Normal Eq. (Raw Scale)   GD (Scaled)
               Intercept             -286.853401 207183.649091
           Median_Income            38717.982131  71292.629582
              Median_Age              850.470609  12229.977163
               Tot_Rooms               -4.548680  -1164.666653
            Tot_Bedrooms               85.658054  29732.769111
              Population              -45.339914 -47282.561159
              Households               73.571900  22880.669803
                Latitude           -57687.638954 -15007.133608
               Longitude           -16526.166308 -28845.676784
       Distance_to_coast               -0.275192 -24941.166486
          Distance_to_LA               -0.166111 -26355.558470
    Distance_to_SanDiego                0.413439   -990.014038
     Distance_to_SanJose                0.152918  -4728.549749
Dis

,Feature,Normal Eq. (Raw Scale),GD (Scaled)
0,Intercept,-286.853401,207183.649091
1,Median_Income,38717.982131,71292.629582
2,Median_Age,850.470609,12229.977163
3,Tot_Rooms,-4.548680,-1164.666653
4,Tot_Bedrooms,85.658054,29732.769111
5,Population,-45.339914,-47282.561159
6,Households,73.571900,22880.669803
7,Latitude,-57687.638954,-15007.133608
8,Longitude,-16526.166308,-28845.676784
9,Distance_to_coast,-0.275192,-24941.166486


In [18]:


# =========================================================
# VI. REGULARIZATION TUNING & PLOTTING (Manual Implementation)
# =========================================================

print("\n" + "="*60)
print("  REGULARIZATION TUNING (Ridge and Lasso)")
print("="*60)

# Define a range of lambda values on a log scale for tuning
lambda_values = np.logspace(-4, 4, 30) # 30 points from 10^-4 to 10^4

# Set training parameters
# Note: You may need to tune learning_rate or n_iterations if models don't converge
learning_rate = 0.01
n_iterations = 5000

ridge_validation_errors = []
lasso_validation_errors = []

# --- 1. RIDGE REGRESSION TUNING ---
for lambda_val in lambda_values:
    # Train the model on the Training Set
    ridge_weights = ridge_gradient_descent(
        X_train_with_bias_scaled, T_train_col, 
        lambda_reg=lambda_val,
        learning_rate=learning_rate,
        n_iterations=n_iterations
    )
    
    # Predict and evaluate on the Validation Set
    T_validation_predict_ridge = X_validation_with_bias_scaled @ ridge_weights
    mse_val = calculate_mse(T_validation_col, T_validation_predict_ridge)
    ridge_validation_errors.append(mse_val)

# --- 2. LASSO REGRESSION TUNING ---
for lambda_val in lambda_values:
    # Train the model on the Training Set
    lasso_weights = lasso_gradient_descent(
        X_train_with_bias_scaled, T_train_col, 
        lambda_reg=lambda_val,
        learning_rate=learning_rate,
        n_iterations=n_iterations
    )
    
    # Predict and evaluate on the Validation Set
    T_validation_predict_lasso = X_validation_with_bias_scaled @ lasso_weights
    mse_val = calculate_mse(T_validation_col, T_validation_predict_lasso)
    lasso_validation_errors.append(mse_val)

# Find optimal lambda
optimal_lambda_ridge = lambda_values[np.argmin(ridge_validation_errors)]
optimal_lambda_lasso = lambda_values[np.argmin(lasso_validation_errors)]

print(f"Optimal Lambda for Ridge: {optimal_lambda_ridge:.4e} (Min Validation MSE: {np.min(ridge_validation_errors):,.2f})")
print(f"Optimal Lambda for Lasso: {optimal_lambda_lasso:.4e} (Min Validation MSE: {np.min(lasso_validation_errors):,.2f})")


  REGULARIZATION TUNING (Ridge and Lasso)
Optimal Lambda for Ridge: 1.0000e-04 (Min Validation MSE: 4,628,477,244.35)
Optimal Lambda for Lasso: 1.0000e-04 (Min Validation MSE: 4,628,477,235.93)


In [19]:
# =========================================================
# V-B. GRADIENT DESCENT USING SCIKIT-LEARN
# =========================================================

print("\n" + "="*60)
print("       3. GRADIENT DESCENT (SCIKIT-LEARN SGDRegressor)")
print("="*60)

"""
SGDRegressor uses Stochastic Gradient Descent (SGD) by default,
which updates weights after each sample (or mini-batch).
We configure it to behave more like Batch GD for fair comparison.

Key parameters:
- max_iter: maximum number of epochs (passes through dataset)
- learning_rate: 'constant' with eta0 to match our learning rate
- penalty: None for standard linear regression (no regularization)
- tol: tolerance for stopping criterion
"""

# Initialize SGDRegressor
sgd_regressor = SGDRegressor(
    max_iter=2000,           # Match our n_iterations
    learning_rate='constant', # Use constant learning rate
    eta0=0.01,               # Match our learning_rate
    penalty=None,            # No regularization (standard linear regression)
    random_state=42,         # For reproducibility
    tol=1e-6                 # Stopping tolerance
)

# Train on scaled training data (without bias term - sklearn adds it automatically)
sgd_regressor.fit(X_train_scaled, T_train_col.ravel())

# Get coefficients (sklearn handles bias separately)
W_sklearn = np.concatenate([[sgd_regressor.intercept_[0]], sgd_regressor.coef_])

print(f"\nTraining completed in {sgd_regressor.n_iter_} iterations")

# --- Predict on Validation and Test Sets ---
T_validation_predict_sklearn = sgd_regressor.predict(X_validation_scaled).reshape(-1, 1)
T_test_predict_sklearn = sgd_regressor.predict(X_test_scaled).reshape(-1, 1)

print("\nScikit-Learn SGD predictions completed.")


       3. GRADIENT DESCENT (SCIKIT-LEARN SGDRegressor)

Training completed in 10 iterations

Scikit-Learn SGD predictions completed.


In [20]:
# =========================================================
# V-C. COMPREHENSIVE COMPARISON OF ALL METHODS
# =========================================================

print("\n" + "="*80)
print("           COMPREHENSIVE MODEL COMPARISON")
print("="*80)

# --- 1. Compare Coefficients ---
feature_names = ['Intercept'] + list(X_train_raw.columns)
coefficients_df_full = pd.DataFrame({
    'Feature': feature_names,
    'Normal Eq. (Raw Scale)': W_direct_sol.flatten(),
    'Manual GD (Scaled)': W_gd.flatten(),
    'Scikit-Learn SGD (Scaled)': W_sklearn.flatten()
})

print("\n" + "="*80)
print("           COEFFICIENT COMPARISON - ALL METHODS")
print("="*80)
print("Note: Normal Eq. weights apply to raw features.")
print("      Manual GD and Scikit-Learn weights apply to scaled features.")
print("\n" + coefficients_df_full.to_string(index=False))

# --- 2. Compare Metrics on Validation Set ---
print("\n" + "="*80)
print("         VALIDATION SET METRICS COMPARISON")
print("="*80)
validation_metrics = compare_metrics_all(
    T_validation_col, 
    T_validation_prediction, 
    T_validation_predict_gd, 
    T_validation_predict_sklearn,
    "Validation Set"
)

# --- 3. Compare Metrics on Test Set ---
print("\n" + "="*80)
print("         TEST SET METRICS COMPARISON")
print("="*80)
test_metrics = compare_metrics_all(
    T_test_col, 
    T_test_prediction, 
    T_test_predict_gd,
    T_test_predict_sklearn,
    "Test Set"
)

# --- 4. Summary Statistics ---
print("\n" + "="*80)
print("           SUMMARY & INSIGHTS")
print("="*80)

# Calculate coefficient differences
coef_diff_manual_sklearn = np.abs(W_gd.flatten() - W_sklearn.flatten())
max_coef_diff = np.max(coef_diff_manual_sklearn)
avg_coef_diff = np.mean(coef_diff_manual_sklearn)

print(f"\nCoefficient Comparison (Manual GD vs Scikit-Learn SGD):")
print(f"  - Maximum difference: {max_coef_diff:.6f}")
print(f"  - Average difference: {avg_coef_diff:.6f}")

print(f"\nKey Observations:")
print(f"  - All three methods should produce similar results on scaled data")
print(f"  - Small differences may occur due to:")
print(f"    * Different convergence criteria")
print(f"    * Numerical precision")
print(f"    * Implementation details (batch vs stochastic updates)")

# Display final table
print("\n" + "="*80)
print("           FINAL COEFFICIENTS TABLE")
print("="*80)
display(coefficients_df_full)


           COMPREHENSIVE MODEL COMPARISON

           COEFFICIENT COMPARISON - ALL METHODS
Note: Normal Eq. weights apply to raw features.
      Manual GD and Scikit-Learn weights apply to scaled features.

                 Feature  Normal Eq. (Raw Scale)  Manual GD (Scaled)  Scikit-Learn SGD (Scaled)
               Intercept             -286.853401       207183.649091              212668.021608
           Median_Income            38717.982131        71292.629582               79967.466777
              Median_Age              850.470609        12229.977163               16572.166193
               Tot_Rooms               -4.548680        -1164.666653               -9561.538270
            Tot_Bedrooms               85.658054        29732.769111               45210.507365
              Population              -45.339914       -47282.561159              -46851.624198
              Households               73.571900        22880.669803               27940.166474
                Latitude

,Feature,Normal Eq. (Raw Scale),Manual GD (Scaled),Scikit-Learn SGD (Scaled)
0,Intercept,-286.853401,207183.649091,212668.021608
1,Median_Income,38717.982131,71292.629582,79967.466777
2,Median_Age,850.470609,12229.977163,16572.166193
3,Tot_Rooms,-4.548680,-1164.666653,-9561.538270
4,Tot_Bedrooms,85.658054,29732.769111,45210.507365
5,Population,-45.339914,-47282.561159,-46851.624198
6,Households,73.571900,22880.669803,27940.166474
7,Latitude,-57687.638954,-15007.133608,-94701.884182
8,Longitude,-16526.166308,-28845.676784,-61189.048093
9,Distance_to_coast,-0.275192,-24941.166486,-10363.212785
